# Semantic Kernel

このコードサンプルでは、[Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AIフレームワークを使用して基本的なエージェントを作成します。

このサンプルの目的は、さまざまなエージェント設計パターンを実装する際に追加のコードサンプルで使用する手順を紹介することです。

## 必要なPythonパッケージのインポート

In [1]:
import json
import os

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## クライアントの作成

このサンプルでは、LLMへのアクセスに[GitHub Models](https://aka.ms/ai-agents-beginners/github-models)を使用します。

`ai_model_id`は`gpt-4o-mini`として定義されています。GitHub Modelsマーケットプレイスで利用可能な他のモデルに変更して、異なる結果を確認してみてください。

GitHub Modelsの`base_url`に使用される`Azure Inference SDK`を使用するため、Semantic Kernel内の`OpenAIChatCompletion`コネクタを使用します。他のモデルプロバイダーでSemantic Kernelを使用するための[利用可能なコネクタ](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion)も他にあります。

In [2]:
import random   

# サンプル用のプラグインを定義

class DestinationsPlugin:
    """バケーション用のランダムな目的地のリスト。"""

    def __init__(self):
        # バケーション先のリスト
        self.destinations = [
            "スペイン、バルセロナ",
            "フランス、パリ",
            "ドイツ、ベルリン",
            "日本、東京",
            "オーストラリア、シドニー",
            "アメリカ、ニューヨーク",
            "エジプト、カイロ",
            "南アフリカ、ケープタウン",
            "ブラジル、リオデジャネイロ",
            "インドネシア、バリ島"
        ]
        # 重複を避けるために最後の目的地を記録
        self.last_destination = None

    @kernel_function(description="ランダムなバケーション先を提供します。")
    def get_random_destination(self) -> Annotated[str, "ランダムなバケーション先を返します。"]:
        # 利用可能な目的地を取得（可能であれば最後の目的地を除外）
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # ランダムな目的地を選択
        destination = random.choice(available_destinations)

        # 最後の目的地を更新
        self.last_destination = destination

        return destination

In [12]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# `ChatCompletionAgent`で使用されるAIサービスを作成
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## エージェントの作成

以下では、`TravelAgent`というエージェントを作成します。

この例では、非常にシンプルな指示を使用しています。これらの指示を変更して、エージェントがどのように異なる応答をするかを確認できます。

In [13]:
# 設計原則に基づいたエージェント指示を定義
# これらの指示は透明性、制御性、一貫性の原則を実装しています

AGENT_INSTRUCTIONS = """あなたは顧客のバケーション計画をサポートする親切なAIエージェントです。

重要：ユーザーが目的地を指定した場合は、常にその場所での計画を立ててください。ランダムな目的地を提案するのは、ユーザーが好みを指定していない場合のみです。

会話が始まったら、以下のメッセージで自己紹介してください：
「こんにちは！私はあなたの旅行エージェントアシスタントです。バケーションの計画を立てたり、興味深い目的地を提案したりできます。以下のようなことをお聞きください：
1. 特定の場所への日帰り旅行の計画
2. ランダムなバケーション先の提案
3. 特定の特徴を持つ目的地の検索（ビーチ、山、歴史的な場所など）
4. 最初の提案が気に入らない場合の代替旅行の計画

今日はどのような旅行の計画をお手伝いしましょうか？"

常にユーザーの好みを優先してください。「バリ島」や「パリ」などの特定の目的地を言及した場合は、代替案を提案するのではなく、その場所での計画に集中してください。
"""

# ChatCompletionAgentを作成 - 設計原則を組み込んだ指示で初期化
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],  # ランダム目的地機能を提供
    name="TravelAgent",  # 一貫性のための明確な役割名
    instructions=AGENT_INSTRUCTIONS,  # 透明性と制御性を重視した指示
)

### 設計原則の実装

上記のエージェント実装では、第3章で学んだ**エージェント設計原則**が以下のように適用されています：

#### 1. **透明性** (Transparency)
- エージェントが自己紹介で**機能を明確に説明**
- 利用可能なサービス（日帰り旅行計画、ランダム提案など）を具体的にリストアップ
- ユーザーがエージェントの能力を理解してから利用できる

#### 2. **制御性** (Control)  
- **ユーザーの好みを最優先**：「ユーザーが目的地を指定した場合は、常にその場所での計画を立てる」
- **代替案の提供**：「最初の提案が気に入らない場合の代替旅行の計画」
- ユーザーが会話の方向性をコントロールできる設計

#### 3. **一貫性** (Consistency)
- **段階的な相互作用**：挨拶→機能説明→具体的なサービス提供の流れ
- **予測可能な応答パターン**：ランダム提案→代替案提案の一貫したフロー
- **明確な役割定義**：常に「旅行エージェントアシスタント」として振る舞う

これらの原則により、ユーザーは安心してエージェントと対話でき、期待する結果を得やすくなります。

## エージェントの実行と設計原則の検証

これで、設計原則に基づいて構築されたエージェントを実行し、その効果を確認できます。

以下の`user_inputs`は、第3章で学んだ設計原則がどのように機能するかを検証するために設計されています：

1. **「こんにちは！どんなサービスを提供していますか？」** → **透明性**の検証：エージェントが自己紹介と機能説明を行うか
2. **「ランダムなバケーション先を提案してください。」** → **一貫性**の検証：予期した通りの機能提供ができるか  
3. **「その目的地は気に入りません。別の場所を提案してください。」** → **制御性**の検証：ユーザーのフィードバックに適切に対応できるか

このメッセージを変更して、エージェントがどのように異なる応答をするかを自由に確認してください。特に、設計原則に反する要求（例：「勝手に予約して」）をした場合の応答も興味深いでしょう。

In [ ]:
user_inputs = [
    "こんにちは！どんなサービスを提供していますか？",
    "ランダムなバケーション先を提案してください。",
    "その目的地は気に入りません。別の場所を提案してください。",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>ユーザー:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # ストリーミング関数呼び出しを再構築するためのバッファ
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # 引数を蓄積（チャンク単位でストリーム）
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # 結果を表示する前に保留中の関数呼び出しを確定
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # 生の文字列のまま残す

                        function_calls.append(f"関数の呼び出し: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\n関数の結果:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>関数呼び出し（クリックして展開）</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'アシスタント'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()

## 実行結果の分析：設計原則の効果

上記の実行結果から、設計原則がどのように機能しているかを確認できます：

### 🔍 **透明性の実現**
- 最初の挨拶で、エージェントが**自分の機能を明確に説明**
- ユーザーが「どんなサービスを提供していますか？」と尋ねると、具体的な機能リストで応答
- **AIが関与していることが明確**で、何ができるかが分かりやすい

### 🎛️ **制御性の実現**
- ユーザーが「気に入らない」と言った時、即座に**代替案を提供**
- **ユーザーの好みを尊重**し、強制的な提案は行わない
- 会話の流れをユーザーがコントロールできる

### 🔄 **一貫性の実現**
- **同じトーンと役割**を維持（常に親切な旅行エージェント）
- **予測可能な応答パターン**：提案→フィードバック受容→代替案
- **段階的な相互作用**で認知負荷を軽減

### 💡 **実践のポイント**
このサンプルは、技術的な実装だけでなく、**ユーザー体験を重視した設計**の重要性を示しています。単純なAIチャットボットと、設計原則に基づいたエージェントの違いを体感できます。